# Assignment 4.1 Mood Detection with OpenCV
Name: Zapanta, Jaison Martin<br>
Section: CPE32S8

## Introduction
As an introduction to computer vision, you are to perform the simple task of mood detection. Mood detection is the process of identifying and understanding a person's current emotional state. It can be done through a variety of methods, but we will focus on Facial expression recognition. Facial expressions are one of the most important cues for understanding human emotions. Mood detection systems can use computer vision techniques to analyze facial features and identify specific expressions, such as happiness, sadness, anger, you must attain the following objectives:
* Recognize when a face is yours or not.
* 
Recognize only your mood (happy, sad, angry, or confused)
* 
Perform testing to show the performance of your implementation.

In [1]:
import os
import sys
import cv2
import errno
import numpy as np
from deepface import DeepFace

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [2]:
# Function to read images for training
def read_images(path, sz=None):
    c = 0
    X, y = [], []

    for dirname, dirnames, filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    if(filename == ".directory"):
                        continue
                    filepath = os.path.join(subject_path, filename)
                    im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

                    # Resize the images to the prescribed size
                    if (sz is not None):
                        im = cv2.resize(im, (200,200))

                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(c)

                except IOError as e:
                    print(f"I/O Error({e.errno}): {e.strerror}")
                except:
                    print("Unexpected error:", sys.exc_info()[0])
                    raise
            c = c+1
    return [X, y]

In [3]:
# Function for mood detection
def detect_mood():
    names = ['Zapanta']
    [X, y] = read_images(r"pics/Zapanta", (200,200))
    y = np.asarray(y, dtype=np.int32)

    # Train the LBPH algorithm
    lbph = cv2.face.LBPHFaceRecognizer_create()
    lbph.train(X, y)

    camera = cv2.VideoCapture(0)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, 720)

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            # Perform face recognition
            roi_gray = gray[y:y+h, x:x+w]
            params = lbph.predict(roi_gray)
            confidence = params[1]
            if confidence < 75:
                label = names[params[0]]
            else:
                label = "???"
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Perform mood detection
            face = img[y:y+h, x:x+w]
            results = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
            if isinstance(results, list):
                for result in results:
                    emotion = max(result['emotion'].items(), key=lambda x: x[1])[0]
                    cv2.putText(img, "{} ({})".format(label, emotion), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
            else:
                emotion = max(results['emotion'].items(), key=lambda x: x[1])[0]
                cv2.putText(img, "{} ({})".format(label, emotion), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

        cv2.imshow("Mood Detection", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

In [5]:
detect_mood()

### Detecting my face and my mood

The algorithm used, DeepFace, does not include the mood for "Confused," therefore there is no output for that specific mood. I considered training my own algorithm that includes it, but after much consideration, I figured that I may not have enough time for it.

Happy
<img src="happy.jpg">

Sad
<img src="sad.jpg">

Angry
<img src="angry.jpg">

### Detecting other's face and other mood

Although it cannot detect "Confused," DeepFace can detect other emotions such as:

Surprised
<img src="surprise.jpg">

Fear
<img src="fear.jpg">

Disgust
<img src="disgust.jpg">